# Model Build
 - Tensorflow로 딥러닝 모델을 구축하는 방법 정리
 - Keras API를 활용
 - 1) Sequential API
 - 2) Functional API
 - 3) Subclassing (Custom Model)


----------
아래와 같은 모델을 구현한다고 생각해보자

**(input: 784-dimensional vectors)**

       ↓
**[Dense (64 units, relu activation)]**

       ↓
**[Dense (64 units, relu activation)]**

       ↓
**[Dense (10 units, softmax activation)]**

       ↓
**(output: logits of a probability distribution over 10 classes)**

-----------

### 1. Sequential API
 - 가장 간단한 구현방법
 - 모델의 층이 순차적으로 구성되어있지 않거나, 여러 입출력을 갖는 모델의 경우는 사용하기 어려울 수 있다

In [1]:
# Sequential API
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


# 구현 방법이 간단
# 모델의 층들이 순차적이지 않은 경우, 여러개의 input, output 구조를 가지는 경우는 다른 방법 사용

### 2. Funtional API
 - keras에 구현되어 있는 layer를 활용하는 방법

 - 특히, 
     - **다중 입력, 출력**값 모델
     - **shared layers**를 활용하는 모델
     - **데이터 흐름이 순차적이지 않은** 모델
 
 위와 같은 모델을 구현할 때는 Funtional API나 Subclassing 방법을 사용하는 것이 적절

In [2]:
# Functional API
# 입력값을 받는 Input layer가 필요하다
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(32, ))
hidden1 = layers.Dense(64, activation='relu')(inputs)
hidden2 = layers.Dense(64, activation='relu')(hidden1)
outputs = layers.Dense(10, activation='softmax')(hidden2)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [3]:
# Custom Layer
# 여러 레이어를 하나로 묶은 레이어가 필요하거나,
# Keras 패키지에 없는 새로운 연산을 하는 레이어가 필요한 경우 커스텀 레이어 정의
# layers 패키지의 Layer를 상속받음
class CustomLayer(layers.Layer):
    
    # 객체 생성시 호출되는 메서드
    def __init__(self, hidden_dim, hidden_dim2, output_dim):
        super(CustomLayer, self).__init__()
        self.hidden_dim = hidden_dim
        self.hidden_dim2 = hidden_dim2
        self.output_dim = output_dim
    
    # init 메서드에서 정의해도 됨. 취향차이
    def build(self, input_shape):
        self.hidden_layer1 = layers.Dense(self.hidden_dim, activation='relu')
        self.hidden_layer2 = layers.Dense(self.hidden_dim2, activation='relu')
        self.output = layers.Dense(self.output_dim, activaiton='softmax')
    
    # __call__ 이 아니라 call
    def call(self, inputs):
        hidden1 = self.hidden_layer1(inputs)
        hidden2 = self.hidden_layer2(hidden1)
        return self.output(hidden2)
    # keras Layer를 Callbable 객체로 만들면 보다 정확하게는 __call__()이 내부적으로 호출이 됩니다.
    # __call__()은 부모 Layer에 감춰져 있기에 작성해줄 필요가 없으며,
    # __call__()이 해주는 것은 build()를 호출한 뒤에 call()을 호출하는 역할을 합니다.
    # 참고링크: https://www.inflearn.com/questions/199313

    
model = tf.keras.Sequential()
model.add(CustomLayer(64, 64, 10))

### 3. Subclassing (Custom Model)
 - 모델 내부 연산들을 직접구현하는 방법으로 자유도가 가장 높다

In [4]:
# Custom Model
# keras의 Model 클래스를 상속받아 구현
class mymodel(tf.keras.Model):
    def __init__(self, hidden_dim, hidden_dim2, output_dim):
        super(mymodel, self).__init__()
        self.hidden_layer1 = layer.Dense(self.hidden_dim, activation='relu')
        self.hidden_layer2 = layer.Dense(self.hidden_dim2, activation='relu')
        self.output = layer.Dense(self.output_dim, activation='softmax')
        
    def call(self, inputs):
        hidden1 = self.hidden_layer1(inputs)
        hidden2 = self.hidden_layer2(hidden1)
        return self.output
    
    # 파이토치 모델 구현방법과도 비슷

#### < 참고자료 >
 - 케라스 The Functional API 문서 [(Link)](https://keras.io/guides/functional_api/)
 - 텐서플로2와 머신러닝으로 시작하는 자연어처리